In [ ]:
import pandas as pd
from libs import preprocessa, modelos, vo_X, metricas
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
random_state=26

In [ ]:
#Se estiver rodando localmente 
caminhoDados="dados/"

#Esse json é gerado no notebook TrataSegmentosAnotam.ipynb
df  = pd.read_json(caminhoDados+'dfAlvoAnota.json') 
vocabulario = df["textoTotal"]


In [ ]:
#filtrar as classes mais relevantes a partir de 0.5% de presença
classesInteresse = [1,3,9,22,24,13,10,4,25]

#União Alt e inc
df =  df[df['idTipoAnotacao'].isin(classesInteresse)]
df.loc[df['idTipoAnotacao'] == 3, 'idTipoAnotacao'] = 1

#dfInteresse.groupby(['idTipoAnotacao','nomeTipoAnotacao']).idSegmento.count()


In [ ]:
##Atributos iniciais
y = df['idTipoAnotacao']

dadosOriginais = []

vocabulario = df["textoIntegraLimpo"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["textoIntegraLimpo"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"Segmento"))

vocabulario = df["verbosTextoIntegra"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["verbosTextoIntegra"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"VerbosSegmento"))

vocabulario = df["textoTotal"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["textoTotal"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"Segmento+Ementa"))

vocabulario = df["verbos"]
x = preprocessa.vetorizaTFIDF(vocabulario,df["verbos"].astype(str))
dadosOriginais.append(vo_X.vo_X(x,y,"TodosVerbos"))


In [ ]:
## Seleção Atributos
dadosChi = []
for dados in dadosOriginais:
    xChi1 = preprocessa.seleciona(vocabulario,dados.x, y,0.1)
    dadosChi.append(vo_X.vo_X(xChi1,y, dados.estrategia + " Chi 0.1"))
    
    xChi25 = preprocessa.seleciona(vocabulario,dados.x, y,0.25)
    dadosChi.append(vo_X.vo_X(xChi25,y, dados.estrategia + " Chi 0.25"))
    
    xChi50 = preprocessa.seleciona(vocabulario,dados.x, y,0.5)
    dadosChi.append(vo_X.vo_X(xChi50,y, dados.estrategia + " Chi 0.5"))
    
   

In [ ]:
#Reamostragem Smote
smote = SMOTE(random_state=random_state)
dadosSmote = []
for dados in dadosOriginais:     
    xs, ys = smote.fit_sample(dados.x,y)
    dadosSmote.append(vo_X.vo_X(xs, ys, dados.estrategia + " Smote"))
    
for dados in dadosChi:     
    xs, ys = smote.fit_sample(dados.x,y)
    dadosSmote.append(vo_X.vo_X(xs, ys, dados.estrategia + " Smote"))

In [ ]:
#Reamostragem NearMiss
sampler = NearMiss()
dadosNearMiss = []

for dados in dadosOriginais: 
    xn, yn = sampler.fit_resample(dados.x,y)
    dadosNearMiss.append(vo_X.vo_X(xn, yn, dados.estrategia + " NearMiss"))

for dados in dadosChi:
    xn, yn = sampler.fit_resample(dados.x,y)
    dadosNearMiss.append(vo_X.vo_X(xn, yn, dados.estrategia + " NearMiss"))


In [ ]:
listaDados = [*dadosOriginais, *dadosChi, *dadosSmote, *dadosNearMiss]

In [ ]:
resultadoGeral = []
for dados in listaDados:
    classificadores = modelos.classificadores()
    classificadores.LSVC = True
    classificadores.MNB = True
    resultadosClassificacao =  classificadores.classificar(dados.x,dados.y)
    for res in resultadosClassificacao:
        res.insert(1, dados.estrategia) 
    resultadoGeral.extend(resultadosClassificacao)   

In [ ]:
df_resultado  = pd.DataFrame(resultadoGeral)
df_resultado.columns = ['Modelo','Estratégia',
                        'Acurácia %','DP','Precisão',
                        'DP', 'Revocação','DP', 'F1','DP' ]
df_resultado.loc[df_resultado['Modelo']=='LinearSVC','Modelo']='SVM'
df_resultado.loc[df_resultado['Modelo']=='MultinomialNB','Modelo']='Naive Bayes'
df_resultado.sort_values(['F1'], ascending = False)

In [ ]:
df_resultado.to_csv("resultado.csv")

In [ ]:
print(df_resultado.sort_values(['F1'], ascending = False).to_latex(index=False))